In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
n_gpus = len(os.environ['CUDA_VISIBLE_DEVICES'].split(','))
model_name = 'ception_8classifier_trained'

trainval_split_ratio = 0.88
train_batch_size =16

import keras.backend as K
import tensorflow as tf

# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)
# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

Using TensorFlow backend.


# 导入必要的库

In [2]:
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import l2

from keras.preprocessing.image import *

import matplotlib.pyplot as plt
import random
import os
import cv2
from tqdm import tqdm
from glob import glob
import multiprocessing

from sklearn.model_selection import train_test_split
from collections import Counter
from keras import backend as K
from keras.utils import multi_gpu_model

from IPython.display import display, Image

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# 读取数据集

In [3]:
df = pd.read_csv('../train/Annotations/train.csv', header=None)
df.columns = ['filename', 'label_name', 'label']
df = df.sample(frac=1).reset_index(drop=True) # shuffle

df.label_name = df.label_name.str.replace('_labels', '')

display(df.head())
c = Counter(df.label_name)
c

,filename,label_name,label
0,Images/collar_design_labels/c120f26a3dbfaeb602...,collar_design,ynnnn
1,Images/coat_length_labels/2eafe3fabc96c20f6eb9...,coat_length,nnnnnnyn
2,Images/skirt_length_labels/2db264dcce17ab2e332...,skirt_length,nynnnn
3,Images/lapel_design_labels/63e18c4274595aa011d...,lapel_design,nnynn
4,Images/lapel_design_labels/5aeb6e3a5dd7704f93d...,lapel_design,nnnny


Counter({'coat_length': 11320,
         'collar_design': 8393,
         'lapel_design': 7034,
         'neck_design': 5696,
         'neckline_design': 17148,
         'pant_length': 7460,
         'skirt_length': 19333,
         'sleeve_length': 13299})

In [4]:
label_count = dict([(x, len(df[df.label_name == x].label.values[0])) for x in c.keys()])
label_names = list(label_count.keys())
display(label_count)

{'coat_length': 8,
 'collar_design': 5,
 'lapel_design': 5,
 'neck_design': 5,
 'neckline_design': 10,
 'pant_length': 6,
 'skirt_length': 6,
 'sleeve_length': 9}

## 生成 y

In [5]:
fnames = df['filename'].values
#width = 331
width = 299
n = len(df)
y = [np.zeros((n, label_count[x])) for x in label_count.keys()]
for i in range(n):
    label_name = df.label_name[i]
    label = df.label[i]
    y[label_names.index(label_name)][i, label.find('y')] = 1

## 读取图片

In [6]:
def f(index):
    return index, cv2.resize(cv2.imread('../train/'+fnames[index]), (width, width))

X = np.zeros((n, width, width, 3), dtype=np.uint8)
with multiprocessing.Pool(30) as pool:
    with tqdm(pool.imap_unordered(f, range(n)), total=n) as pbar:
        for i, img in pbar:
            X[i] = img[:,:,::-1]

100%|██████████| 89683/89683 [03:15<00:00, 458.01it/s]


In [7]:
n_train = int(n*trainval_split_ratio)

X_train = X[:n_train]
X_valid = X[n_train:]
y_train = [x[:n_train] for x in y]
y_valid = [x[n_train:] for x in y]

In [8]:
def display_images(imgs, w=8, h=4, figsize=(24, 12)):
    plt.figure(figsize=figsize)
    for i in range(w*h):
        plt.subplot(h, w, i+1)
        plt.imshow(imgs[i])

In [9]:
class Generator():
    def __init__(self, X, y, batch_size=32, aug=False):
        def generator():
            idg = ImageDataGenerator(horizontal_flip=True,
                                     rotation_range=20,
                                     zoom_range=0.15)
            while True:
                for i in range(0, len(X), batch_size):
                    X_batch = X[i:i+batch_size].copy()
                    y_barch = [x[i:i+batch_size] for x in y]
                    if aug:
                        for j in range(len(X_batch)):
                            X_batch[j] = idg.random_transform(X_batch[j])
                    yield X_batch, y_barch
        self.generator = generator()
        self.steps = len(X) // batch_size + 1

In [10]:
gen_train = Generator(X_train, y_train, batch_size=train_batch_size, aug=True)

# 数据集探索

## 类别分布

In [11]:
# plt.figure(figsize=(26, 14))
# for i in range(8):
#     plt.subplot(2, 4, i+1)
#     counts = Counter(y[i].argmax(axis=-1)[np.where(y[i].any(axis=-1))])
#     pd.Series(counts).plot('bar')

### 抽样可视化

In [12]:
# plt.figure(figsize=(26, 14))
# w = 8
# h = 4
# for i in range(w*h):
#     plt.subplot(h, w, i+1)
#     index = np.random.randint(n)
#     plt.title(str([y[x][index].argmax() if y[x][index].any() else -1 for x in range(8) ]))
#     plt.imshow(X[index])
#     plt.axis('off')

# 搭建模型并训练

## 搭建模型

In [13]:
def acc(y_true, y_pred):
    index = tf.reduce_any(y_true > 0.5, axis=-1)
    res = tf.equal(tf.argmax(y_true, axis=-1), tf.argmax(y_pred, axis=-1))
    index = tf.cast(index, tf.float32)
    res = tf.cast(res, tf.float32)
    return tf.reduce_sum(res * index) / (tf.reduce_sum(index) + 1e-7)

In [14]:
base_model = InceptionResNetV2(weights='imagenet', input_shape=(width, width, 3), include_top=False, pooling='avg')
#InceptionResNetV2,Xception
input_tensor = Input((width, width, 3))
x = input_tensor
x = Lambda(nasnet.preprocess_input)(x)
x = base_model(x)
x = Dropout(0.5)(x)
x = [Dense(count, activation='softmax', name=name)(x) for name, count in label_count.items()]

model = Model(input_tensor, x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

from IPython.display import SVG
#from keras.utils.vis_utils import model_to_dot, plot_model

#plot_model(model, show_shapes=True, to_file='model_simple.png')
#SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [15]:
#model2 = multi_gpu_model(model, n_gpus)
model2 = model

In [16]:
opt = SGD(1e-2, momentum=0.9, nesterov=True, decay=0.01/40)

In [17]:
model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=[acc])
model2.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=4, validation_data=(X_valid, y_valid))

Epoch 1/4
5317/5317 [==============================] - 2388s 449ms/step - loss: 0.9098 - collar_design_loss: 0.0866 - coat_length_loss: 0.1155 - skirt_length_loss: 0.1398 - lapel_design_loss: 0.0704 - sleeve_length_loss: 0.1423 - neck_design_loss: 0.0722 - neckline_design_loss: 0.2223 - pant_length_loss: 0.0607 - collar_design_acc: 0.4753 - coat_length_acc: 0.5541 - skirt_length_acc: 0.7397 - lapel_design_acc: 0.4404 - sleeve_length_acc: 0.5716 - neck_design_acc: 0.3199 - neckline_design_acc: 0.5570 - pant_length_acc: 0.5072 - val_loss: 0.5492 - val_collar_design_loss: 0.0655 - val_coat_length_loss: 0.0655 - val_skirt_length_loss: 0.0856 - val_lapel_design_loss: 0.0379 - val_sleeve_length_loss: 0.0714 - val_neck_design_loss: 0.0529 - val_neckline_design_loss: 0.1303 - val_pant_length_loss: 0.0401 - val_collar_design_acc: 0.5347 - val_coat_length_acc: 0.6402 - val_skirt_length_acc: 0.8196 - val_lapel_design_acc: 0.5286 - val_sleeve_length_acc: 0.6813 - val_neck_design_acc: 0.3749 - val_

In [18]:
model2.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=[acc])
model2.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=2, validation_data=(X_valid, y_valid))

Epoch 1/2
5317/5317 [==============================] - 2461s 463ms/step - loss: 0.4188 - collar_design_loss: 0.0419 - coat_length_loss: 0.0583 - skirt_length_loss: 0.0663 - lapel_design_loss: 0.0299 - sleeve_length_loss: 0.0607 - neck_design_loss: 0.0363 - neckline_design_loss: 0.0952 - pant_length_loss: 0.0302 - collar_design_acc: 0.6192 - coat_length_acc: 0.7083 - skirt_length_acc: 0.8621 - lapel_design_acc: 0.5862 - sleeve_length_acc: 0.7659 - neck_design_acc: 0.4709 - neckline_design_acc: 0.7882 - pant_length_acc: 0.6042 - val_loss: 0.4551 - val_collar_design_loss: 0.0458 - val_coat_length_loss: 0.0551 - val_skirt_length_loss: 0.0723 - val_lapel_design_loss: 0.0321 - val_sleeve_length_loss: 0.0659 - val_neck_design_loss: 0.0399 - val_neckline_design_loss: 0.1032 - val_pant_length_loss: 0.0408 - val_collar_design_acc: 0.6444 - val_coat_length_acc: 0.6901 - val_skirt_length_acc: 0.8610 - val_lapel_design_acc: 0.5950 - val_sleeve_length_acc: 0.7223 - val_neck_design_acc: 0.4518 - val_

In [19]:
model2.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=[acc])
model2.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=3, validation_data=(X_valid, y_valid))

Epoch 1/3
5317/5317 [==============================] - 2476s 466ms/step - loss: 0.2155 - collar_design_loss: 0.0219 - coat_length_loss: 0.0309 - skirt_length_loss: 0.0311 - lapel_design_loss: 0.0137 - sleeve_length_loss: 0.0301 - neck_design_loss: 0.0192 - neckline_design_loss: 0.0521 - pant_length_loss: 0.0165 - collar_design_acc: 0.6841 - coat_length_acc: 0.7773 - skirt_length_acc: 0.9198 - lapel_design_acc: 0.6377 - sleeve_length_acc: 0.8372 - neck_design_acc: 0.5302 - neckline_design_acc: 0.8614 - pant_length_acc: 0.6509 - val_loss: 0.3674 - val_collar_design_loss: 0.0440 - val_coat_length_loss: 0.0511 - val_skirt_length_loss: 0.0455 - val_lapel_design_loss: 0.0239 - val_sleeve_length_loss: 0.0496 - val_neck_design_loss: 0.0341 - val_neckline_design_loss: 0.0919 - val_pant_length_loss: 0.0274 - val_collar_design_acc: 0.6594 - val_coat_length_acc: 0.7103 - val_skirt_length_acc: 0.9024 - val_lapel_design_acc: 0.6118 - val_sleeve_length_acc: 0.7688 - val_neck_design_acc: 0.4907 - val_

In [20]:
model2.compile(optimizer=Adam(1e-6), loss='categorical_crossentropy', metrics=[acc])
model2.fit_generator(gen_train.generator, steps_per_epoch=gen_train.steps, epochs=1, validation_data=(X_valid, y_valid))

Epoch 1/1
5317/5317 [==============================] - 2476s 466ms/step - loss: 0.1037 - collar_design_loss: 0.0102 - coat_length_loss: 0.0137 - skirt_length_loss: 0.0129 - lapel_design_loss: 0.0055 - sleeve_length_loss: 0.0147 - neck_design_loss: 0.0100 - neckline_design_loss: 0.0286 - pant_length_loss: 0.0082 - collar_design_acc: 0.7167 - coat_length_acc: 0.8227 - skirt_length_acc: 0.9499 - lapel_design_acc: 0.6649 - sleeve_length_acc: 0.8710 - neck_design_acc: 0.5655 - neckline_design_acc: 0.9016 - pant_length_acc: 0.6772 - val_loss: 0.3744 - val_collar_design_loss: 0.0448 - val_coat_length_loss: 0.0545 - val_skirt_length_loss: 0.0397 - val_lapel_design_loss: 0.0241 - val_sleeve_length_loss: 0.0518 - val_neck_design_loss: 0.0363 - val_neckline_design_loss: 0.0961 - val_pant_length_loss: 0.0270 - val_collar_design_acc: 0.6553 - val_coat_length_acc: 0.7101 - val_skirt_length_acc: 0.9174 - val_lapel_design_acc: 0.6226 - val_sleeve_length_acc: 0.7744 - val_neck_design_acc: 0.4909 - val_

# 计算验证集准确率

In [21]:
model.save('B_model_%s.h5' % model_name)

In [22]:
y_pred = model2.predict(X_valid, batch_size=128, verbose=1)
a = np.array([x.any(axis=-1) for x in y_valid]).T.astype('uint8')
b = [np.where((a == np.eye(8)[x]).all(axis=-1))[0] for x in range(8)]
for c in range(8):
    y_pred2 = y_pred[c][b[c]].argmax(axis=-1)
    y_true2 = y_valid[c][b[c]].argmax(axis=-1)
    print(label_names[c], (y_pred2 == y_true2).mean())

15247/15247 [==============================] - 169s 11ms/step
collar_design 0.860259032038
coat_length 0.86307277628
skirt_length 0.946342925659
lapel_design 0.906688687036
sleeve_length 0.885052248978
neck_design 0.821503131524
neckline_design 0.866824805151
pant_length 0.892744479495


In [23]:
counts = Counter(df.label_name)
counts

Counter({'coat_length': 11320,
         'collar_design': 8393,
         'lapel_design': 7034,
         'neck_design': 5696,
         'neckline_design': 17148,
         'pant_length': 7460,
         'skirt_length': 19333,
         'sleeve_length': 13299})

In [24]:
s = 0
n = 0
for c in range(8):
    y_pred2 = y_pred[c][b[c]].argmax(axis=-1)
    y_true2 = y_valid[c][b[c]].argmax(axis=-1)
    s += counts[label_names[c]] * (y_pred2 == y_true2).mean()
    n += counts[label_names[c]]
print(s / n)

0.887985584357


# 在测试集上预测

In [25]:
df_test = pd.read_csv('../test/z_rank/Tests/question.csv', header=None)
df_test.columns = ['filename', 'label_name', 'label']

fnames_test = df_test.filename

n_test = len(df_test)
df_test.head()

,filename,label_name,label
0,Images/collar_design_labels/faad3490a16c7f3d4f...,collar_design_labels,?
1,Images/collar_design_labels/0b2b4254f35ce3a41a...,collar_design_labels,?
2,Images/collar_design_labels/7f2be608e06f804dd5...,collar_design_labels,?
3,Images/collar_design_labels/4b09d4dca80caac42e...,collar_design_labels,?
4,Images/collar_design_labels/de91f00a05e84d7239...,collar_design_labels,?


In [27]:
def f(index):
    return index, cv2.resize(cv2.imread('../test/z_rank/'+fnames_test[index]), (width, width))

X_test = np.zeros((n_test, width, width, 3), dtype=np.uint8)
with multiprocessing.Pool(12) as pool:
    with tqdm(pool.imap_unordered(f, range(n_test)), total=n_test) as pbar:
        for i, img in pbar:
            X_test[i] = img[:,:,::-1]

100%|██████████| 15042/15042 [01:18<00:00, 190.50it/s]


In [28]:
y_pred = model2.predict(X_test, batch_size=128, verbose=1)

15042/15042 [==============================] - 161s 11ms/step


In [29]:
for i in range(n_test):
    problem_name = df_test.label_name[i].replace('_labels', '')
    problem_index = label_names.index(problem_name)
    probs = y_pred[problem_index][i]
    df_test.label[i] = ';'.join(np.char.mod('%.8f', probs))

In [30]:
fname_csv = 'B_test_%s.csv' % model_name
df_test.to_csv(fname_csv, index=None, header=None)